In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.functions import *


In [2]:
from pyspark.shell import spark

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.5.1
      /_/

Using Python version 3.12.2 (tags/v3.12.2:6abddd9, Feb  6 2024 21:26:36)
Spark context Web UI available at http://DESKTOP-5E7MJHC:4040
Spark context available as 'sc' (master = local[*], app id = local-1712737701636).
SparkSession available as 'spark'.


In [3]:
df = spark.read.load('googleplaystore.csv', format='csv',sep=',',header='true',escape='"',inferschema='true')

In [4]:
df.count()

10841

In [5]:
df.show(1)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|      Genres|   Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M| 10,000+|Free|    0|      Everyone|Art & Design|January 7, 2018|      1.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
only showing top 1 row



Schema

In [6]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



Data Cleaning   

In [7]:
df=df.drop("size","Content Rating","Last Updated","Android Ver")

In [8]:
df.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+-----------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|Current Ver|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+-----------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|        Art & Design|      1.0.0|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|500,000+|Free|    0|Art & Design;Pret...|      2.0.0|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+-----------+
only showing top 2 rows



In [9]:
df=df.drop("Current Ver")

In [10]:
df.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|500,000+|Free|    0|Art & Design;Pret...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 2 rows



In [11]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [12]:
from pyspark.sql.functions import regexp_replace, col

df=df.withColumn("Reviews",col("Reviews").cast(IntegerType()))\
.withColumn("Installs",regexp_replace(col("Installs"),"[^0-9]",""))\
.withColumn("Installs",col("Installs").cast(IntegerType()))\
            .withColumn("Price",regexp_replace(col("Price"),"[$]",""))\
                .withColumn("Price",col("Price").cast(IntegerType()))

 

In [13]:
df.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 2 rows



In [14]:
df.createOrReplaceTempView("apps")

In [15]:
spark.sql("select * from apps").show()

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|   50000|Free|    0|        Art & Design|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178|   50000|Free|    0|        Art & Design|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 1000000

Top reviews

In [16]:
spark.sql("select App,sum(Reviews) from apps group by 1 order by 2 desc").show()

+--------------------+------------+
|                 App|sum(Reviews)|
+--------------------+------------+
|           Instagram|   266241989|
|  WhatsApp Messenger|   207348304|
|      Clash of Clans|   179558781|
|Messenger – Text ...|   169932272|
|      Subway Surfers|   166331958|
|    Candy Crush Saga|   156993136|
|            Facebook|   156286514|
|         8 Ball Pool|    99386198|
|        Clash Royale|    92530298|
|            Snapchat|    68045010|
|     Viber Messenger|    56675481|
|UC Browser - Fast...|    53140694|
|             YouTube|    51278853|
|        Temple Run 2|    48710930|
|Sniper 3D Gun Sho...|    46022233|
|      My Talking Tom|    44668928|
|Duolingo: Learn L...|    44047832|
|       Google Photos|    43423827|
|Clean Master- Spa...|    42916526|
|                 Pou|    41939801|
+--------------------+------------+
only showing top 20 rows



Top 10 installed Apps

In [17]:
spark.sql("select App,sum(Installs) from apps group by 1 order by 2 desc").show()

+--------------------+-------------+
|                 App|sum(Installs)|
+--------------------+-------------+
|      Subway Surfers|   6000000000|
|            Hangouts|   4000000000|
|           Instagram|   4000000000|
|        Google Drive|   4000000000|
|         Google News|   4000000000|
|       Google Photos|   4000000000|
|    Candy Crush Saga|   3500000000|
|  WhatsApp Messenger|   3000000000|
|Messenger – Text ...|   3000000000|
|Google Chrome: Fa...|   3000000000|
|               Gmail|   3000000000|
|Maps - Navigate &...|   3000000000|
|Skype - free IM &...|   3000000000|
|        Temple Run 2|   3000000000|
|     Viber Messenger|   2500000000|
|             YouTube|   2000000000|
|   Google Play Games|   2000000000|
|                 Pou|   2000000000|
|  Google Street View|   2000000000|
|              Google|   2000000000|
+--------------------+-------------+
only showing top 20 rows



Category wise Distribution

In [18]:
spark.sql("select Category,sum(Installs) from apps group by 1 order by 2 desc").show()

+-------------------+-------------+
|           Category|sum(Installs)|
+-------------------+-------------+
|               GAME|  35086024415|
|      COMMUNICATION|  32647276251|
|       PRODUCTIVITY|  14176091369|
|             SOCIAL|  14069867902|
|              TOOLS|  11452771915|
|             FAMILY|  10258263505|
|        PHOTOGRAPHY|  10088247655|
| NEWS_AND_MAGAZINES|   7496317760|
|   TRAVEL_AND_LOCAL|   6868887146|
|      VIDEO_PLAYERS|   6222002720|
|           SHOPPING|   3247848785|
|      ENTERTAINMENT|   2869160000|
|    PERSONALIZATION|   2325494782|
|BOOKS_AND_REFERENCE|   1921469576|
|             SPORTS|   1751174498|
| HEALTH_AND_FITNESS|   1583072512|
|           BUSINESS|   1001914865|
|            FINANCE|    876648734|
|          EDUCATION|    871452000|
|MAPS_AND_NAVIGATION|    724281890|
+-------------------+-------------+
only showing top 20 rows



Top Paid apps

In [19]:
spark.sql("select App,sum(Price) from apps where Type='Paid' group by 1 order by 2 desc").show()

+--------------------+----------+
|                 App|sum(Price)|
+--------------------+----------+
|I'm Rich - Trump ...|       400|
|   I Am Rich Premium|       399|
|  I AM RICH PRO PLUS|       399|
|I'm Rich/Eu sou R...|       399|
|      I am Rich Plus|       399|
|most expensive ap...|       399|
|       I Am Rich Pro|       399|
|  I am rich(premium)|       399|
|           I am Rich|       399|
|          I am Rich!|       399|
|         💎 I'm rich|       399|
|I am rich (Most e...|       399|
|           I am rich|       399|
|         Eu Sou Rico|       394|
|           I Am Rich|       389|
| I am extremely Rich|       379|
|       I am rich VIP|       299|
|        EP Cook Book|       200|
|Vargo Anesthesia ...|       158|
|       cronometra-br|       154|
+--------------------+----------+
only showing top 20 rows

